# ACM Abstract Topic Classifier
MiniLM embedding + LogisticRegression (One‑vs‑Rest)

In [1]:
# --- Imports & environment ---
import os, pickle, joblib, subprocess
from pathlib import Path

import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords

import torch
from sentence_transformers import SentenceTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score, hamming_loss)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

nltk.download('stopwords')
stop_words = stopwords.words('english')

MODELDIR = Path('models')
MODELDIR.mkdir(exist_ok=True)


/home/konstanty/Projects/UW/UWvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on: cuda


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/konstanty/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# --- Helper Functions ---
def flatten_if_single(x):
    """If x is a list of length 1, return its first element."""
    if isinstance(x, list) and len(x) == 1:
        return x[0]
    return x

def load_or_train_mlb(train_labels, path=Path('models/mlb_model.pkl'), all_labels=None):
    if path.exists():
        print('✓ MLB loaded')
        return joblib.load(path)
    print('… training MultiLabelBinarizer')
    if all_labels is None:
        all_labels = sorted({lbl for sub in train_labels for lbl in sub})
    mlb = MultiLabelBinarizer(classes=all_labels)
    mlb.fit(train_labels)
    joblib.dump(mlb, path)
    return mlb

def notify():
    try:
        subprocess.run(['play', '-nq', '-t', 'alsa', 'synth', '0.3', 'sine', '1000'])
    except FileNotFoundError:
        pass


In [3]:
# --- Load data ---
notebook_dir = os.getcwd()
test_path  = os.path.join(notebook_dir, 'data', 'DM2023_test_docs.tsv')
train_path = os.path.join(notebook_dir, 'data', 'DM2023_training_docs_and_labels.tsv')

test = pd.read_csv(test_path,  sep='\t', encoding='latin1',
                   header=None, names=['Textfile','Text','Topics'])

train_full = pd.read_csv(train_path, sep='\t', encoding='latin1',
                         header=None, names=['Textfile','Text','Topics'])

# Split topics string into list
train_full['Topics'] = (train_full['Topics']
                        .apply(flatten_if_single)
                        .str.split(r'\s*,\s*'))


In [4]:
# --- Train / Validation split ---
split = int(train_full.shape[0] * 0.8)
val   = train_full.iloc[split:].reset_index(drop=True)
train = train_full.iloc[:split].reset_index(drop=True)
print('Train:', train.shape, ' Val:', val.shape)


Train: (80000, 3)  Val: (20000, 3)


In [5]:
# --- SentenceTransformer Embeddings ---
bert_path = Path('models/minilm_model.pkl')  # change filename if you switch model

if bert_path.exists():
    print('\n\t model loaded')
    model = joblib.load(bert_path)
else:
    print('\n\t… loading MiniLM')
    model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
    joblib.dump(model, bert_path)

embed_paths = [Path('models/X_train_embed.npy'),
               Path('models/X_val_embed.npy'),
               Path('models/X_test_embed.npy')]

if all(p.exists() for p in embed_paths):
    X_train_embed = np.load(embed_paths[0])
    X_val_embed   = np.load(embed_paths[1])
    X_test_embed  = np.load(embed_paths[2])
else:
    X_train_embed = model.encode(train['Text'].tolist(), batch_size=32, show_progress_bar=True)
    X_val_embed   = model.encode(val['Text'].tolist(),   batch_size=32, show_progress_bar=True)
    X_test_embed  = model.encode(test['Text'].tolist(),  batch_size=32, show_progress_bar=True)

    np.save(embed_paths[0], X_train_embed)
    np.save(embed_paths[1], X_val_embed)
    np.save(embed_paths[2], X_test_embed)



	 model loaded


In [6]:
# --- Label binarization ---
all_topics = sorted({lbl for sub in train_full['Topics'] for lbl in sub})
mlb = load_or_train_mlb(train['Topics'], all_labels=all_topics)

y_train = mlb.transform(train['Topics'])
y_val   = mlb.transform(val['Topics'])


✓ MLB loaded


/home/konstanty/Projects/UW/UWvenv/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator MultiLabelBinarizer from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
# --- Train or load classifier ---
clf_path = Path('models/classifier.pkl')

if clf_path.exists():
    print('\n\tFound classifier model!')
    with open(clf_path, 'rb') as f:
        clf = pickle.load(f)
else:
    print('\n\tWe need to train classifier first...')
    # base = LogisticRegression(penalty='l2', 
    #                           C=1.0, 
    #                           dual=False, 
    #                           class_weight='balanced',
    #                           solver='saga', 
    #                           max_iter=300,
    #                           random_state=42)
    base = LinearSVC(C=1.0, dual=False)     # dual=False szybsze dla n_samples > n_features
    clf = OneVsRestClassifier(base, n_jobs=-1)
    clf.fit(X_train_embed, y_train)
    joblib.dump(clf, clf_path)
    notify()



	We need to train classifier first...


In [9]:
# --- Validation ---
# y_pred_bin = clf.predict(X_val_embed)  # default threshold
# y_pred_bin = (clf.decision_function(X_val_embed) > -0.3).astype(int)


scores = clf.decision_function(X_val_embed)
y_pred_bin = np.zeros_like(scores, dtype=int)

for i, row in enumerate(scores):
    top_idx = row.argmax()
    y_pred_bin[i, top_idx] = 1                 # zawsze co najmniej 1 etykieta
    y_pred_bin[i, row > -0.3] = 1          # + inne, które przekroczą próg


val['PredictedTopics'] = mlb.inverse_transform(y_pred_bin)

y_val_true_bin = mlb.transform(val['Topics'])

print('\n\n=== SAMPLE-BASED METRICS ===')
print('Precision (samples):', precision_score(y_val_true_bin, y_pred_bin,
                                             average='samples', zero_division=0))
print('Recall    (samples):', recall_score(y_val_true_bin, y_pred_bin, average='samples'))
print('F1        (samples):', f1_score(y_val_true_bin, y_pred_bin,
                                       average='samples', zero_division=0))
print('Subset accuracy   :', accuracy_score(y_val_true_bin, y_pred_bin))
print('Hamming loss      :', hamming_loss(y_val_true_bin, y_pred_bin))




=== SAMPLE-BASED METRICS ===
Precision (samples): 0.5559144047619047
Recall    (samples): 0.3821262518037518
F1        (samples): 0.4141946558996559
Subset accuracy   : 0.06345
Hamming loss      : 0.0073371508379888265


In [12]:
# --- Predict on test & save submission ---
# y_test_bin = clf.predict(X_test_embed)
# 

scores = clf.decision_function(X_test_embed)
y_test_bin = np.zeros_like(scores, dtype=int)

for i, row in enumerate(scores):
    top_idx = row.argmax()
    y_test_bin[i, top_idx] = 1                 # zawsze co najmniej 1 etykieta
    y_test_bin[i, row > -0.3] = 1          # + inne, które przekroczą próg

label_lists = mlb.inverse_transform(y_test_bin)

submission = test
submission["Topics"] = label_lists #.str.join(",")
submission.to_csv('submission.csv', index=False)
notify()